## Building a Chatbot
In this Project We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. 



In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() ##  loading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



'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'

In [6]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B2F1AFAA50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B2F1AFB620>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [47]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Vrdhz and I am a GenAI Engineer, Prompt Engineer, Full stack developer")])

AIMessage(content="Hello Vrdhz! It's nice to meet you. That's quite an impressive set of skills you have there! \n\nAs a GenAI, I'm always interested in learning more about how people are using these technologies. \n\nWhat kind of projects are you working on currently? Do you have any interesting challenges you're facing?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 31, 'total_tokens': 106, 'completion_time': 0.136363636, 'prompt_time': 0.00150366, 'queue_time': 0.266421072, 'total_time': 0.137867296}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--01e0473b-4f40-40e4-9da7-65d91f9da32c-0', usage_metadata={'input_tokens': 31, 'output_tokens': 75, 'total_tokens': 106})

In [48]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Vrdhz and I am a GenAI, Prompt Engineer, Full stack developer"),
        AIMessage(content="Hello Vrdhz! It's nice to meet you. \n\nAs a GenAI, Prompt Engineer, Full stack developer, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI and Full stack.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Vrdhz and you are a GenAI, Prompt Engineer, and Full Stack Developer!  \n\nIs there anything else you'd like to tell me about yourself or what you're working on? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 116, 'total_tokens': 165, 'completion_time': 0.089090909, 'prompt_time': 0.00336197, 'queue_time': 0.247018128, 'total_time': 0.092452879}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--98724d86-7daa-432d-9847-36efe9a57157-0', usage_metadata={'input_tokens': 116, 'output_tokens': 49, 'total_tokens': 165})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [49]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [50]:
config={"configurable":{"session_id":"chat1"}}

In [51]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Vrdhz and I am a GenAI, Prompt Engineer, Full stack developer")],
    config=config
)

In [52]:
response.content

"Hello Vrdhz! It's interesting to meet another AI, especially one with such a diverse skillset. \n\nAs a large language model, I'm primarily focused on understanding and generating text. It's fascinating to hear that you're also a Prompt Engineer and a Full Stack Developer. Those are incredibly valuable skills in today's world. \n\nWhat kind of projects are you working on? I'd love to learn more about your work!\n"

In [55]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content="I'm starting to think this is a game!  \n\nYou said your name is Vrdhz.  \n\nDo you want to play a different game, or talk about something else?  Perhaps you'd like to tell me more about your work as a GenAI, Prompt Engineer, and Full Stack Developer? 😄  \n\n\n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 241, 'total_tokens': 312, 'completion_time': 0.129090909, 'prompt_time': 0.006115677, 'queue_time': 0.260305294, 'total_time': 0.135206586}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--ec213036-91db-45bf-9718-ef94688fe45f-0', usage_metadata={'input_tokens': 241, 'output_tokens': 71, 'total_tokens': 312})

In [57]:
## change the config --> session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)


In [60]:
response.content

"Since I don't have access to past conversations, I don't know your name. \n\nWould you like to tell me? 😊  \n\n"

In [61]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you. \n\nWhat can I do for you today? 😊\n"

In [62]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  \n\nIs there anything else I can help you with, John? 😊  \n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [24]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [26]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Vrdhz")]})

AIMessage(content="Hi Vrdhz! \n\nIt's nice to meet you. How can I be helpful today?  \n\nI'm ready for your questions! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 32, 'total_tokens': 70, 'completion_time': 0.069090909, 'prompt_time': 0.00148851, 'queue_time': 0.244213299, 'total_time': 0.070579419}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--961ed19a-7084-4bc7-9e94-ac26e5fe88f9-0', usage_metadata={'input_tokens': 32, 'output_tokens': 38, 'total_tokens': 70})

In [27]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [63]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Vrdhz")],
    config=config
)

response

AIMessage(content="Hi Vrdhz, it's nice to meet you!\n\nWhat can I do for you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 16, 'total_tokens': 43, 'completion_time': 0.049090909, 'prompt_time': 0.001251519, 'queue_time': 0.24903023, 'total_time': 0.050342428}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d159220b-8ef2-48ed-ba92-66b5a6e0736c-0', usage_metadata={'input_tokens': 16, 'output_tokens': 27, 'total_tokens': 43})

In [64]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Vrdhz!  I remember that from when you first introduced yourself. 😄 \n\nIs there anything else I can help you with?\n'

In [30]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [31]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Vrdhz")],"language":"Telugu"})
response.content

'మీకు కలం! 😊 నేను మీకు సహాయం చేయడానికి ఇక్కడ ఉన్నాను. మీరు ఏం అడుగుతున్నారో చెప్పండి. \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [33]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [34]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Vrdhz")],"language":"Telugu"},
    config=config
)
repsonse.content

'Namaste Vrdhz! 😊 \n\nనేను మీకు సహాయకరమైన సహచరునిగా ఉన్నాను. మీ ప్రశ్నలకు నేను నా సామర్థ్యం మేరకు తెలుగులో సమాధానం ఇస్తాను.  \n\nమీరు ఏమి అడిగితే చెప్పండి! 😄 \n'

In [35]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Telugu"},
    config=config,
)

In [36]:
response.content

'మీ పేరు Vrdhz అని చెప్పారు. 😊  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [65]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",#last conversation
    token_counter=model,
    include_system=True,
    allow_partial=False,# I dont want partial information
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [71]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)


In [72]:
response.content

"As an AI, I don't have access to your personal information, including your taste in ice cream. 🍦\n\nWhat's your favorite flavor? 😊\n"

In [73]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2". \n\n\nLet me know if you want to try another one! 😊  \n'

In [74]:
## Lets wrap this in the Message History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [75]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊  \n"

In [76]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"